In [1]:
import torchvision
from torchvision.models import quantization
torchvision.__version__

/home/aiteam/miniconda3/envs/loader/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'0.13.0+cu113'

In [2]:
import os
import torch

def print_model_size(mdl):
    torch.save(mdl.state_dict(), "tmp.pt")
    print("%.2f MB" %(os.path.getsize("tmp.pt")/1e6))
    os.remove('tmp.pt')

In [3]:
# 기존의 ResNet50 Weight를 가져오는 것도 가능한데 성능 차이가 있을 듯
from torchvision.models import ResNet50_Weights, resnet50
standard_model = resnet50(weights=ResNet50_Weights.auto)

quantized_model = quantization.resnet50(weights=quantization.ResNet50_QuantizedWeights.auto, 
                                         quantize=True)

/home/aiteam/miniconda3/envs/loader/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/aiteam/miniconda3/envs/loader/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_QuantizedWeights.IMAGENET1K_FBGEMM_V1`. You can also use `weights=ResNet50_QuantizedWeights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
print_model_size(standard_model)
print_model_size(quantized_model)

102.54 MB
26.16 MB


In [5]:
model_dynamic_quantized = torch.quantization.quantize_dynamic(
    standard_model, qconfig_spec={torch.nn.Linear}, dtype=torch.qint8
)

In [6]:
print_model_size(model_dynamic_quantized)

96.40 MB


In [7]:
import torch

# define a floating point model
class M(torch.nn.Module):
    def __init__(self):
        super(M, self).__init__()
        self.fc = torch.nn.Linear(4, 4)

    def forward(self, x):
        x = self.fc(x)
        return x

# create a model instance
model_fp32 = M()
# create a quantized model instance
model_int8 = torch.quantization.quantize_dynamic(
    model_fp32,  # the original model
    {torch.nn.Linear},  # a set of layers to dynamically quantize
    dtype=torch.qint8)  # the target dtype for quantized weights


In [8]:
# run the model
input_fp32 = torch.randn(4, 4, 4, 4)

In [9]:
input_fp32.dtype

torch.float32

In [10]:
res = model_int8(input_fp32)

In [11]:
res.dtype

torch.float32

In [16]:
%%timeit -r 10 -n 3
input_fp32 = torch.randn(4, 4, 4, 4)
res = model_fp32(input_fp32)

The slowest run took 19.82 times longer than the fastest. This could mean that an intermediate result is being cached.
139 µs ± 208 µs per loop (mean ± std. dev. of 10 runs, 3 loops each)


In [17]:
%%timeit -r 10 -n 3
input_fp32 = torch.randn(4, 4, 4, 4)
res = model_int8(input_fp32)

The slowest run took 7.51 times longer than the fastest. This could mean that an intermediate result is being cached.
253 µs ± 282 µs per loop (mean ± std. dev. of 10 runs, 3 loops each)
